# Imports

In [3]:
# IMPORTS
import copy
import numpy as np
from enum import Enum
import bisect 
import time

# CLASS INITIALIZATIONS

In [4]:
class Direction(Enum):
  HORIZONTAL = 1
  VERTICAL = 2

class Coordinate():
  def __init__(self, X, Y):
    self.X = X 
    self.Y = Y
  # Primarily for testing to see if Coordinate works
  def __str__(self):
    return "(" + str(self.X) + ", " + str(self.Y) + ")"

#
# CAR
# 

class Car(object):
  def __init__(self, letter, coordinate, fuel):
    self.letter = letter  # Remember that A is the most important
    self.coordinates = []
    self.coordinates.append(coordinate)
    self.fuel = int(fuel)

  def isA(self):
    return self.letter == 'A'

  def move(self, amt):
    if self.direction == Direction.HORIZONTAL:
      for c in self.coordinates:
        c.Y += amt
    else:
      for c in self.coordinates:
        c.X += amt
    self.fuel -= abs(amt)
  
  def canMove(self):
    return self.fuel > 0

  def compareLetters(self, letter):
    return self.letter == letter

  def addCoordinate(self, newCoord):
    self.coordinates.append(newCoord)
    # Update direction
    # L-R (Horizontal)
    if self.coordinates[0].X == newCoord.X and self.coordinates[0].Y != newCoord.Y:
      self.direction = Direction.HORIZONTAL
    # U-D (Vertical)
    else:
      self.direction = Direction.VERTICAL

  def getLetter(self):
    return self.letter; 

  # Primarily for testing to see if Car works
  def __str__(self):
    s = self.letter + " -> ";
    for i in self.coordinates:
      s += str(i)
    return s + " | " + str(self.fuel) + " | " + str(self.direction)

  @staticmethod
  def create(letter, coordinates, fuel):
    return Car(letter, coordinates, fuel)


#
# BOARD
# 

class Board(object):
  def __init__(self, cars):
    self.grid = [['.' for i in range(6)] for j in range(6)]
    self.size = {'X': 6, 'Y': 6}
    self.cars = []
    for car in cars:  
      c = car
      self.cars.append(copy.deepcopy(c));
      for coord in c.coordinates:
        self.grid[coord.X][coord.Y] = car.letter
        
  def __eq__(self, obj):
        return self.grid == obj.grid

  def findCarPaths(self, car):
    if car.canMove():
      fwd = self.checkPathForward(car)
      bwd = self.checkPathBackward(car)
      carPath = bwd + fwd
    else:
      carPath = []
    return carPath
  
  def moveCar(self, carLetter, amt):
    for car in self.cars: 
      if car.letter == carLetter:
        for coord in car.coordinates:
          #print(coord)
          self.grid[coord.X][coord.Y] = "."
        car.move(amt)
        for coord in car.coordinates:
          #print(coord)
          self.grid[coord.X][coord.Y] = carLetter

        # Is this a regular car that just left?
        if car.letter == self.grid[2][5] and car.direction == Direction.HORIZONTAL and car.letter != "A":
          for coord in car.coordinates:
            self.grid[coord.X][coord.Y] = "."
          self.cars.remove(car)  
         

  # ......
  # ...AA. (1,3)(1,4)
  # ......
  # startPoint = 4
  # range(5,6) -> 5
  # next gridPos = (1,5)
  # append -> 5 - 4 = 1

  # ...D.. (0,3)
  # ...D.. (1,3)
  # ......
  # startPoint = 1
  # range(2,6) -> 2, 3, 4, 5
  # next gridPos = (2,3)
  # append -> 2 - 1 = 1
  def checkPathForward(self, car):
    dir = car.direction
    lastCoord = car.coordinates[len(car.coordinates)-1]
    if dir == Direction.HORIZONTAL:
      startPoint = lastCoord.Y
    else:
      startPoint = lastCoord.X

    possibleMoves = []
    for i in range(startPoint+1, 6):
        #fuel req
      if(car.fuel < i-startPoint):
        continue;
      # Find next position to check    
      if dir == Direction.HORIZONTAL:
        gridPos = self.grid[lastCoord.X][i] 
      else:
        gridPos = self.grid[i][lastCoord.Y]

      # If there's an open position, add and continue
      # Else, there is something blocking the way and there's no way to continue any more
      if gridPos == ".":
        possibleMoves.append(i-startPoint)
      else:
        break;

    return possibleMoves

  # ......
  # ...AA. (1,3)(1,4)
  # ......
  # startPoint = 3
  # range(3,0,-1) -> 3, 2, 1
  # i = i - 1 -> 2, 1, 0
  # next gridPos = (1,2)
  # append -> 2 - 3 = -1

  # ...D.. (0,3)
  # ...D.. (1,3)
  # ......
  # startPoint = 0
  # range(0,0) -> NONE
  # next gridPos = NONE
  def checkPathBackward(self, car):
    dir = car.direction
    lastCoord = car.coordinates[0]
    if dir == Direction.HORIZONTAL:
      startPoint = lastCoord.Y
    else:
      startPoint = lastCoord.X

    possibleMoves = []
    for i in range(startPoint, 0, -1):
      if (car.fuel <= (startPoint-i)):
        continue
      # Find next position to check
      if dir == Direction.HORIZONTAL:
        gridPos = self.grid[lastCoord.X][i-1] 
      else:
        gridPos = self.grid[i-1][lastCoord.Y]

      # If there's an open position, add and continue
      # Else, there is something blocking the way and there's no way to continue any more
      if gridPos == ".":
        possibleMoves.append(i-startPoint-1)
      else:
        break;

    return possibleMoves

  def winConditionMet(self):
    if self.grid[2][5] == "A":
      return True      
    return False

  def __str__(self): 
    s = ""
    for x in range(6):
      for y in range(6):
        s += self.grid[x][y]
      s += "\n"
      
    return s


  #Heuristic definitions:

  #h1 = The number of blocking vehicles
  def h1(self):
        count = 0
        last = " "
        for i in range(6):
            e = self.grid[2][5-i]
            if e == "A":
                break
            elif e == last:
                continue
            elif e != ".":
                count += 1
                last = e
        return count
  
  #h2 = The number of blocked positions
  def h2(self):
        count = 0
        for i in range(6):
            e = self.grid[2][5-i]
            if e == "A":
                break
            elif e != ".":
                count += 1
        return count
    
  #h3 = The number of blocking vehicles (h1) multiplied by a constant
  def h3(self, constant):
        return self.h1()*constant
    
  #h4 = Is there a car blocking the exit square (3f)
  #Yes: h4 = 1
  #No:  h4 = 0

  def h4(self):
        if self.grid[2][5] != "." and self.grid[2][5] != "A":
            return 1
        else:
            return 0

# Extracting Data From Sample-input.txt

In [5]:
demos = []
demoFuels = []

with open('sample-input.txt') as f:
    for line in f:
      if not line.startswith(('#', '\n')):
        in_arr = np.array([line.strip()])
        data = np.char.split(in_arr)[0]
        demos.append(data)
f.close()

print(demos)

[['BBIJ....IJCC..IAAMGDDK.MGH.KL.GHFFL.'], ['..I...BBI.K.GHAAKLGHDDKLG..JEEFF.J..'], ['JBBCCCJDD..MJAAL.MFFKL.N..KGGN.HH...'], ['BBB..MCCDD.MAAKL.MJ.KLEEJ.GG..JHHHII', 'J0', 'B4'], ['IJBBCCIJDDL.IJAAL.EEK.L...KFF..GGHH.', 'F0', 'G6'], ['BB.G.HE..G.HEAAG.I..FCCIDDF..I..F...'], ['C.B...C.BHHHAADD........EEGGGF.....F'], ['BB.G.HE..G.HEAAJJ...FCCIDDF..I..F...']]


# Setting the boards up

In [6]:
boards = []
for data in demos:
  # Seperate Grid Data from the Fuel Parameters
  grid = data[0];
  fuels = []
  if len(data) > 1:
    fuels.append(data[1:])

  cars = []
  for row in range(6):
    for col in range(6):
      current = grid[row*6 + col];
      
      # If Unique: Create a car and append
      # If Not Unique: Concat to same letter
      if current == ".":
        continue

      unique = True
      for c in cars:
        if c.compareLetters(current):
          c.addCoordinate(Coordinate(row,col))
          unique = False

      if unique:
        # Now check if there's a unique fuel amount or if its the default 100
        fuelAmt = 100
        if len(fuels) > 0:
          for f in fuels[0]:
            if current == f[0]:
              fuelAmt = f[1:]

        cars.append(Car.create(current, Coordinate(row,col), fuelAmt));

  # Checking if all cars were made properly
  print("CARS: ")
  print("--------")
  for c in cars:
    print(c)
  print()
  
  board = Board(cars)
  # Checking if all boards were made properly
  print("BOARD: ")
  print("--------")
  print(board)
  print()

  boards.append(board)


  for b in boards:
    for c in b.cars:
      print(c)
    print()

CARS: 
--------
B -> (0, 0)(0, 1) | 100 | Direction.HORIZONTAL
I -> (0, 2)(1, 2)(2, 2) | 100 | Direction.VERTICAL
J -> (0, 3)(1, 3) | 100 | Direction.VERTICAL
C -> (1, 4)(1, 5) | 100 | Direction.HORIZONTAL
A -> (2, 3)(2, 4) | 100 | Direction.HORIZONTAL
M -> (2, 5)(3, 5) | 100 | Direction.VERTICAL
G -> (3, 0)(4, 0)(5, 0) | 100 | Direction.VERTICAL
D -> (3, 1)(3, 2) | 100 | Direction.HORIZONTAL
K -> (3, 3)(4, 3) | 100 | Direction.VERTICAL
H -> (4, 1)(5, 1) | 100 | Direction.VERTICAL
L -> (4, 4)(5, 4) | 100 | Direction.VERTICAL
F -> (5, 2)(5, 3) | 100 | Direction.HORIZONTAL

BOARD: 
--------
BBIJ..
..IJCC
..IAAM
GDDK.M
GH.KL.
GHFFL.


B -> (0, 0)(0, 1) | 100 | Direction.HORIZONTAL
I -> (0, 2)(1, 2)(2, 2) | 100 | Direction.VERTICAL
J -> (0, 3)(1, 3) | 100 | Direction.VERTICAL
C -> (1, 4)(1, 5) | 100 | Direction.HORIZONTAL
A -> (2, 3)(2, 4) | 100 | Direction.HORIZONTAL
M -> (2, 5)(3, 5) | 100 | Direction.VERTICAL
G -> (3, 0)(4, 0)(5, 0) | 100 | Direction.VERTICAL
D -> (3, 1)(3, 2) | 100 | D

# Building State Space Search - Uniform Cost Search (UCS)

In [7]:
class Node(object):
  def __init__(self, board, cost):
    self.board = board
    self.cost = cost
    self.path = []
    self.parent = 0
  def __lt__(self, other):
        return self.cost < other.cost
  def __eq__(self, other):
        return self.board == other.board
  
    
class UCS(object):
  def __init__(self):
    self.opened = []
    self.closed = []

  def recursivePrinting(self, node):
    if node.cost != 0:
      self.recursivePrinting(node.parent)  
    print(node.board)

  def search(self, board):
       
    #Initialise first Node with cost 0 (different for other algorithms)
    a = Node(copy.deepcopy(board), 0)
    self.opened.append(a)
    
    start_time = time.time()
    statessearched = 0
      
    while(1):              
        #If open list is empty, exit
        if(len(self.opened)==0):
            print("No Solution")
            return
        
        #Pop the best candidate node
        curr = self.opened.pop(0)
        
        #If the popped node is the solution, exit out and print stuff (later to file)
        if(curr.board.winConditionMet()):
            print("Initial board configuration")
            print()
            print(board)
            print("Car Fuel Available: ", end = '')
            for i in range(len(board.cars)):
                print(""+board.cars[i].letter+":"+str(board.cars[i].fuel)+", ", end ='')
            print()
            
            print("Runtime: %s seconds" % (time.time() - start_time))
            print("Number of moves: ", curr.cost)
            print("Search path length: %s states" % statessearched)

            print("------")
            self.recursivePrinting(curr)

            return


        #print("CHECKING CLOSED + OPEN: ")
        #print("-----")
        #for c in self.closed:
        #  print(c.board)
        #print("-----")

        #for c in self.opened:
        #  print(c.board)
        #print("----- END")

        #If it's not a solution insert into closed list and increment counter
        self.closed.append(curr)
        statessearched+=1
        #print()
        for c in curr.board.cars:
            paths = curr.board.findCarPaths(c)    
            if len(paths) > 0:  
                #print("Car ", c.letter, " can move:", paths)
                
                for i in range(len(paths)):
                    #Create a new node for successor
                    #Increase the cost of this new node (different in other algorithms)
                    newnode = Node(Board(curr.board.cars), curr.cost+1)   
                    newnode.parent = curr                 
                
                    #Apply the move
                    newnode.board.moveCar(c.letter, paths[i])
                    
                    #If the item is already in the open list, skip it
                    #In other algos, need to check that cost is less
                    #For UCS, this will never be the case, since all costs are 1 in this case (breadth first search)
                    if newnode in self.opened or newnode in self.closed:
                        continue

                    #Insert into open list (in sorted order)
                    bisect.insort(self.opened, newnode)
                    


In [31]:
class HNode(object):
  def __init__(self, board, cost, parentCost):
    self.board = board
    self.cost = cost
    self.path = []
    self.parent = 0
    self.parentCost = parentCost
  def __lt__(self, other):
        return self.cost < other.cost
  def __eq__(self, other):
        return self.board == other.board
  def addHeuristic(self, heuristic):
        if heuristic == 1:
            self.cost = self.cost + self.board.h1()
        elif heuristic == 2:
            self.cost = self.cost + self.board.h2()
        elif heuristic == 3:
            self.cost = self.cost + self.board.h3(3)
        elif heuristic == 4:
            self.cost = self.cost + self.board.h4()
        return self.cost
  def setCost(self, cost):
        self.cost = cost
        
        
class GBFS(object):
    def __init__(self):
        self.opened = []
        self.closed = []
    
    def recursivePrinting(self, node):
        if node.cost != 0:
          self.recursivePrinting(node.parent)  
        print(node.board)
        
    def search(self, board, heuristic):
        #Initialise first Node with cost 0
        a = HNode(copy.deepcopy(board), 0, -1)
        
        self.opened.append(a)
        
        start_time = time.time()
        statessearched = 0
      
        while(1):              
            #If open list is empty, exit
            if(len(self.opened)==0):
                print("No Solution")
                return
        
            #Pop the best candidate node
            curr = self.opened.pop(0)
            curr.setCost(curr.parentCost + 1)
        
            #If the popped node is the solution, exit out and print stuff (later to file)
            if(curr.board.winConditionMet()):
                print("Initial board configuration")
                print()
                print(board)
                print("Car Fuel Available: ", end = '')
                for i in range(len(board.cars)):
                    print(""+board.cars[i].letter+":"+str(board.cars[i].fuel)+", ", end ='')
                print()
            
                print("Runtime: %s seconds" % (time.time() - start_time))
                print("Number of moves: ", curr.cost)
                print("Search path length: %s states" % statessearched)

                print("------")
                self.recursivePrinting(curr)

                return

            #If it's not a solution insert into closed list and increment counter
            self.closed.append(curr)
            statessearched+=1
        
            #print()
            for c in curr.board.cars:
                paths = curr.board.findCarPaths(c)    
                if len(paths) > 0:  
                    #print("Car ", c.letter, " can move:", paths)
                    
                    for i in range(len(paths)):
                        #Create a new node for successor
                        newnode = HNode(Board(curr.board.cars), 0, curr.cost)   
                        newnode.parent = curr                 
                
                        #Apply the move
                        newnode.board.moveCar(c.letter, paths[i])
                        newnode.addHeuristic(heuristic)
                    
                        #If the node is already in the open or closed list, skip it
                        if newnode in self.opened or newnode in self.closed:
                            continue

                        #Insert into open list (in sorted order)
                        bisect.insort(self.opened, newnode)

In [32]:
class AAStar(object):
    def __init__(self):
        self.opened = []
        self.closed = []
    
    def recursivePrinting(self, node):
        if node.cost != 0:
          self.recursivePrinting(node.parent)  
        print(node.board)
        
    def search(self, board, heuristic):
        #Initialise first Node with cost 0
        a = HNode(copy.deepcopy(board), 0, -1)
        
        self.opened.append(a)
        
        start_time = time.time()
        statessearched = 0
      
        while(1):              
            #If open list is empty, exit
            if(len(self.opened)==0):
                print("No Solution")
                return
        
            #Pop the best candidate node
            curr = self.opened.pop(0)
            curr.setCost(curr.parentCost + 1)
        
            #If the popped node is the solution, exit out and print stuff (later to file)
            if(curr.board.winConditionMet()):
                print("Initial board configuration")
                print()
                print(board)
                print("Car Fuel Available: ", end = '')
                for i in range(len(board.cars)):
                    print(""+board.cars[i].letter+":"+str(board.cars[i].fuel)+", ", end ='')
                print()
            
                print("Runtime: %s seconds" % (time.time() - start_time))
                print("Number of moves: ", curr.cost)
                print("Search path length: %s states" % statessearched)

                print("------")
                self.recursivePrinting(curr)

                return

            #If it's not a solution insert into closed list and increment counter
            self.closed.append(curr)
            statessearched+=1
        
            #print()
            for c in curr.board.cars:
                paths = curr.board.findCarPaths(c)    
                if len(paths) > 0:  
                    #print("Car ", c.letter, " can move:", paths)
                    
                    for i in range(len(paths)):
                        #Create a new node for successor
                        newnode = HNode(Board(curr.board.cars), curr.cost+1, curr.cost)   
                        newnode.parent = curr                 
                
                        #Apply the move
                        newnode.board.moveCar(c.letter, paths[i])
                        newnode.addHeuristic(heuristic)
                    
                        #If the node is already in the open or closed list, skip if the cost in is higher, else replace
                        if newnode in self.opened:
                            for n in self.opened:
                                if n == newnode and n.cost > newnode.cost:
                                    self.opened.remove(n)
                                    bisect.insort(self.opened, newnode)
                                    break
                            continue
                        elif newnode in self.closed:
                            for n in self.closed:
                                if n == newnode and n.cost > newnode.cost:
                                    self.closed.remove(n)
                                    bisect.insort(self.opened, newnode)
                                    break
                            continue
                        else:
                            #Insert into open list (in sorted order)
                            bisect.insort(self.opened, newnode)

In [28]:
currentBoard = Board(boards[2].cars)
ucs = UCS()
ucs.search(currentBoard)

Initial board configuration

JBBCCC
JDD..M
JAAL.M
FFKL.N
..KGGN
.HH...

Car Fuel Available: J:100, B:100, C:100, D:100, M:100, A:100, L:100, F:100, K:100, N:100, G:100, H:100, 
Runtime: 66.02400612831116 seconds
Number of moves:  17
Search path length: 5239 states
------
JBBCCC
JDD..M
JAAL.M
FFKL.N
..KGGN
.HH...

JBBCCC
JDDL.M
JAAL.M
FFK..N
..KGGN
.HH...

JBBCCC
JDDL.M
JAAL.M
FFK...
..KGGN
.HH..N

JBBCCC
JDDL.M
JAAL.M
FFK...
..KGGN
...HHN

JBBCCC
JDDL.M
JAAL.M
FF....
..KGGN
..KHHN

JBBCCC
JDDL.M
JAAL.M
....FF
..KGGN
..KHHN

.BBCCC
.DDL.M
.AAL.M
J...FF
J.KGGN
J.KHHN

BB.CCC
.DDL.M
.AAL.M
J...FF
J.KGGN
J.KHHN

BB.CCC
DD.L.M
.AAL.M
J...FF
J.KGGN
J.KHHN

BB.CCC
DD.L.M
AA.L.M
J...FF
J.KGGN
J.KHHN

BBKCCC
DDKL.M
AA.L.M
J...FF
J..GGN
J..HHN

BBKCCC
DDKL.M
AA.L.M
J...FF
JGG..N
J..HHN

BBKCCC
DDK..M
AA...M
J..LFF
JGGL.N
J..HHN

BBKCCC
DDK..M
...AAM
J..LFF
JGGL.N
J..HHN

BB.CCC
DDK..M
..KAAM
J..LFF
JGGL.N
J..HHN

BBCCC.
DDK..M
..KAAM
J..LFF
JGGL.N
J..HHN

BBCCCM
DDK..M
..KAA.
J..LFF
JGGL.N
J..HH

In [35]:
currentBoard = Board(boards[2].cars)
gbfs = GBFS()
gbfs.search(currentBoard, 3)

Initial board configuration

JBBCCC
JDD..M
JAAL.M
FFKL.N
..KGGN
.HH...

Car Fuel Available: J:100, B:100, C:100, D:100, M:100, A:100, L:100, F:100, K:100, N:100, G:100, H:100, 
Runtime: 75.94052147865295 seconds
Number of moves:  26
Search path length: 5266 states
------
JBBCCC
JDD..M
JAAL.M
FFKL.N
..KGGN
.HH...

JBBCCC
JDD..M
JAAL.M
FFKL.N
..KGGN
HH....

JBBCCC
JDD..M
JAAL.M
FF.L.N
..KGGN
HHK...

JBBCCC
JDD..M
JAAL.M
.FFL.N
..KGGN
HHK...

.BBCCC
JDD..M
JAAL.M
JFFL.N
..KGGN
HHK...

BB.CCC
JDD..M
JAAL.M
JFFL.N
..KGGN
HHK...

BBCCC.
JDD..M
JAAL.M
JFFL.N
..KGGN
HHK...

BBCCCM
JDD..M
JAAL..
JFFL.N
..KGGN
HHK...

BBCCCM
JDDL.M
JAAL..
JFF..N
..KGGN
HHK...

BBCCCM
JDDL.M
JAAL..
J..FFN
..KGGN
HHK...

BBCCCM
JDDL.M
JAAL..
J.KFFN
..KGGN
HH....

BBCCCM
JDDL.M
JAAL..
J.KFFN
..KGGN
.HH...

BBCCCM
.DDL.M
.AAL..
J.KFFN
J.KGGN
JHH...

BBCCCM
DD.L.M
.AAL..
J.KFFN
J.KGGN
JHH...

BBCCCM
DD.L.M
AA.L..
J.KFFN
J.KGGN
JHH...

BBCCCM
DDKL.M
AAKL..
J..FFN
J..GGN
JHH...

BBCCCM
DDKL.M
AAKL..
JFF..N
J..GGN
JHH..

In [36]:
currentBoard = Board(boards[2].cars)
aastar = AAStar()
aastar.search(currentBoard, 4)

Initial board configuration

JBBCCC
JDD..M
JAAL.M
FFKL.N
..KGGN
.HH...

Car Fuel Available: J:100, B:100, C:100, D:100, M:100, A:100, L:100, F:100, K:100, N:100, G:100, H:100, 
Runtime: 96.77043890953064 seconds
Number of moves:  17
Search path length: 5127 states
------
JBBCCC
JDD..M
JAAL.M
FFKL.N
..KGGN
.HH...

JBBCCC
JDDL.M
JAAL.M
FFK..N
..KGGN
.HH...

JBBCCC
JDDL.M
JAAL.M
FFK...
..KGGN
.HH..N

JBBCCC
JDDL.M
JAAL.M
FFK...
..KGGN
...HHN

JBBCCC
JDDL.M
JAAL.M
FF....
..KGGN
..KHHN

JBBCCC
JDDL.M
JAAL.M
....FF
..KGGN
..KHHN

.BBCCC
.DDL.M
.AAL.M
J...FF
J.KGGN
J.KHHN

BB.CCC
.DDL.M
.AAL.M
J...FF
J.KGGN
J.KHHN

BB.CCC
DD.L.M
.AAL.M
J...FF
J.KGGN
J.KHHN

BB.CCC
DD.L.M
AA.L.M
J...FF
J.KGGN
J.KHHN

BBKCCC
DDKL.M
AA.L.M
J...FF
J..GGN
J..HHN

BBKCCC
DDKL.M
AA.L.M
J...FF
JGG..N
J..HHN

BBKCCC
DDK..M
AA...M
J..LFF
JGGL.N
J..HHN

BBKCCC
DDK..M
...AAM
J..LFF
JGGL.N
J..HHN

BB.CCC
DDK..M
..KAAM
J..LFF
JGGL.N
J..HHN

BBCCC.
DDK..M
..KAAM
J..LFF
JGGL.N
J..HHN

BBCCCM
DDK..M
..KAA.
J..LFF
JGGL.N
J..HH